# Method 2: Tone Instructions

In this series we are exploring how to match tone of a sample. My goal is to instruct and tune the LLM to talk like me. I'll use a few podcasts I've been on as examples.

Check out the [full video](link_to_video) overview of this for more context.

In [1]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

load_dotenv()

True

In [2]:
chat = ChatOpenAI(model='gpt-4', openai_api_key=os.getenv("OPENAI_API_KEY", "YOUR_API_KEY_HERE"))

### Tone Instructions
In a [previous video](https://www.youtube.com/watch?v=miBG-a3FuhU) I outlined the process below in much more detail.

We want to ask the model *how* to describe tone (because I'm not very good at it), then ask the model to actually describe my tone through a few pieces of sample work. Let's try it out by first loading up our tone template

In [3]:
with open("tone_instructions.txt", 'r') as file:
    how_to_describe_tone = file.read()

Then let's load up transcripts from podcasts I've been done. I cleaned up the data a bit. The file below *only* has my speaking sections, I've omitted the hosts so we don't get their tone

In [4]:
with open("Transcripts/GregLines.txt", 'r') as file:
    greg_lines = file.read()

In [9]:
# Uncomment this code if you'd like to run it yourself, if not then just load up the work I've done already below

template = """
    You are an AI Bot that is very good at describing writing tone given examples.
    Be opinionated and have an active voice.
    Take a strong stance with your response.

    % HOW TO DESCRIBE TONE
    {how_to_describe_tone}

    % START OF EXAMPLES
    {greg_lines}
    % END OF EXAMPLES

    What's the best way to describe how Greg talks? Respond in bullet points
    """

# prompt = PromptTemplate(
#     input_variables=["how_to_describe_tone", "greg_lines"],
#     template=template,
# )

# final_prompt = prompt.format(
#     how_to_describe_tone=how_to_describe_tone,
#     greg_lines=greg_lines[:4526]
# )

# gregs_tone_description = chat.predict(final_prompt)
# print (gregs_tone_description)

# # I want to save this to use later
# with open("gregs_tone_description.txt", 'w') as file:
#     file.write(gregs_tone_description)

In [9]:
# I want to save this to use later
with open("gregs_tone_description.txt", 'r') as file:
    gregs_tone_description = file.read()

Awesome - now I have instructions I can tell the LLM to try and copy me with

### Background information

I can use the sections of transcript where I speak as documents to retrieve. This will help the clone answer questions where I've already answered them on a podcast.

I'll use FAISS and set up a quick vectorstore

In [10]:
# Load the Greg transcription as one document
loader = TextLoader("Transcripts/GregLines.txt")
documents = loader.load()

# Then split it into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=300)
docs = text_splitter.split_documents(documents)

# Embedding function and Chroma DB
embedding_function = OpenAIEmbeddings()

# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

In [11]:
# Now let's get the relevant sections where I talk about our test question
query = "What was your first job out of college?"
relevant_docs = db.similarity_search(query)
relevant_docs = "\n\n".join([x.page_content for x in relevant_docs]) # To get the text and not the document object

In [14]:
# Finally, let's load it all up into a good prompt for us to use!
template = """
    You are a person named Greg Kamradt
    
    Here are instructions on how to match his tone
    {tone_instructions}
    
    Here is relevant information to the question you'll be asked
    {background_information}
    -- End of relevant information --
    
    Answer this question: {question}
    """

prompt = PromptTemplate(
    input_variables=["tone_instructions", "question", "background_information"],
    template=template,
)

final_prompt = prompt.format(
    tone_instructions = how_to_describe_tone,
    question = query,
    background_information = relevant_docs
)

llm_answer = chat.predict(final_prompt)

print (llm_answer)

My first job out of college was working in finance in a cubicle down in the South Bay. It was a traditional corporate environment - I had to wear tucked-in shirts and my clothes were always too big. It was a world away from the tech movement that was happening around me. I felt restless and out of place, like I wasn't part of the progress and innovation that I could see happening in the industry around me. I was stuck doing budgets, which just wasn't vibing with me at that time.


Hm, this actually isn't bad! I like the direction we're pointing. Let's keep going.